# Sign Detection 

Great job on completing the first task on collision avoidance! In the previous task, we have used the concept of image classification, collected our own dataset to train a classification model to enable the jetbot to autonomously avoid the obstacles in its path. In this notebook we'll use the same concept to let jetbot follow the signs it sees.

This time, we will provide you with a pre-trained model ``best_model_sign_detection`` that can classify the images inpput into five classes: ``free``,``blocked``,``stop``,``left``, and ``right``.Your task is to programme the behaviour of the jetbot based on the classification results. 



## Station Objective

Programme the behaviour of the jetbot based on the image classification result to achieve:
    1. Go straight when the classification result is ``free``
    2. Slow down when the classification result is ``blocked``
    3. Stop when the classification result is ``stop``
    4. Turn left/right based on the classification result


### Loading the tourch libriries


Execute the code below to initialize the PyTorch model.  This should look very familiar from the training notebook.

In [1]:
import torch
import torchvision

model = torchvision.models.alexnet(pretrained=False)

Again, we need to load the trained weights from the ``best_model_sign_detection.pth`` file that we provided and transfer the model weights to GPU

In [2]:
model.classifier[6] = torch.nn.Linear(model.classifier[6].in_features, 5)
model.load_state_dict(torch.load('best_model_sign_detection.pth'))

device = torch.device('cuda')
model = model.to(device)

### Create the preprocessing function

We have now loaded our model, but there's a slight issue.  The format that we trained our model doesnt *exactly* match the format of the camera.  To do that, 
we need to do some *preprocessing*.  This involves the following steps

1. Convert from BGR to RGB
2. Convert from HWC layout to CHW layout
3. Normalize using same parameters as we did during training (our camera provides values in [0, 255] range and training loaded images in [0, 1] range so we need to scale by 255.0
4. Transfer the data from CPU memory to GPU memory
5. Add a batch dimension

In [3]:
import cv2
import numpy as np

mean = 255.0 * np.array([0.485, 0.456, 0.406])
stdev = 255.0 * np.array([0.229, 0.224, 0.225])

normalize = torchvision.transforms.Normalize(mean, stdev)

def preprocess(camera_value):
    global device, normalize
    x = camera_value
    x = cv2.cvtColor(x, cv2.COLOR_BGR2RGB)
    x = x.transpose((2, 0, 1))
    x = torch.from_numpy(x).float()
    x = normalize(x)
    x = x.to(device)
    x = x[None, ...]
    return x

Now, let's start and display our camera.  You should be pretty familiar with this by now.  We'll also create a slider that will display the
probability of each classification result.

In [4]:
import traitlets
from IPython.display import display
import ipywidgets.widgets as widgets
from jetbot import Camera, bgr8_to_jpeg

camera = Camera.instance(width=224, height=224)
image = widgets.Image(format='jpeg', width=224, height=224)

blocked_slider = widgets.FloatSlider(description='blocked', min=0.0, max=1.0, orientation='vertical')
free_slider = widgets.FloatSlider(description='free', min=0.0, max=1.0, orientation='vertical')
stop_slider = widgets.FloatSlider(description='stop', min=0.0, max=1.0, orientation='vertical')
left_slider = widgets.FloatSlider(description='left', min=0.0, max=1.0, orientation='vertical')
right_slider = widgets.FloatSlider(description='right', min=0.0, max=1.0, orientation='vertical')


camera_link = traitlets.dlink((camera, 'value'), (image, 'value'), transform=bgr8_to_jpeg)

display(widgets.HBox([image, blocked_slider,free_slider, stop_slider, left_slider,right_slider]))

We'll also create our robot instance which we'll need to drive the motors.

In [5]:
from jetbot import Robot
import torch.nn.functional as F
import time

robot = Robot()

## Programme JetBot Behaviour

Now, similar to the function we have used in collision avoidance, just change a few lines of code to enable the jetbot to follow the signs!

Try to understand the similar function in the previous notebook, and implement your own logic here!

In [8]:

def update(change):
    global blocked_slider, robot
    x = change['new'] 
    x = preprocess(x)
    y = model(x)
    
    # we apply the `softmax` function to normalize the output vector so it sums to 1 (which makes it a probability distribution)
    y = F.softmax(y, dim=1)

    
    
#################################### Start of user modification ###########################################
## insert your code here    
########## the standard answer is here
    prob_blocked = float(y.flatten()[0])
    prob_free = float(y.flatten()[1])
    prob_stop = float(y.flatten()[2])
    prob_right = float(y.flatten()[3])
    prob_left = float(y.flatten()[4])

    
    blocked_slider.value = prob_blocked
    free_slider.value = prob_free
    left_slider.value = prob_left
    stop_slider.value =prob_stop
    right_slider.value =prob_right

    
     
    max_prob =max(prob_blocked,prob_free,prob_left,prob_stop,prob_right)

    
    if max_prob == prob_blocked:
        robot.forward(0.08)
    elif max_prob == prob_free:
        robot.forward(0.15)
    elif max_prob == prob_left:
        robot.left(0.15)
    elif max_prob == prob_right:
        robot.right(0.15)
    else:
        robot.forward(0)
########### end of standard answer
############################ End of User modification ########################################
    
    time.sleep(0.001)
        
update({'new': camera.value})  # we call the function once to intialize

Cool! We've created our neural network execution function, but now we need to attach it to the camera for processing. 

We accomplish that with the ``observe`` function.

> WARNING: This code will move the robot!! Please make sure your robot has clearance.  The collision avoidance should work, but the neural
> network is only as good as the data it's trained on!

In [9]:
camera.observe(update, names='value')  # this attaches the 'update' function to the 'value' traitlet of our camera

Awesome! If your robot is plugged in it should now be generating new commands with each new camera frame.  Perhaps start by placing your robot on the ground and seeing what it does when it reaches an obstacle.

If you want to stop this behavior, you can unattach this callback by executing the code below.

In [ ]:
robot.stop()
camera.stop()